In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import csv
import pandas as pd

In [46]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 361.3 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 239.0 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 190.6 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 162.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 272.2 kB/s eta 0:00:00 kB/s eta 0:00:01


In [3]:
html_text = requests.get("https://www.espncricinfo.com/team")
soup = BeautifulSoup(html_text.content,"html.parser")

In [5]:
team_info = soup.find(class_ = "ds-p-0")
semi_links = []

for team in team_info.find_all('a'):
    semi_links.append(team['href'])

In [6]:
team_links = semi_links[0:12]
team_links

['/team/afghanistan-40',
 '/team/australia-2',
 '/team/bangladesh-25',
 '/team/england-1',
 '/team/india-6',
 '/team/ireland-29',
 '/team/new-zealand-5',
 '/team/pakistan-7',
 '/team/south-africa-3',
 '/team/sri-lanka-8',
 '/team/west-indies-4',
 '/team/zimbabwe-9']

In [7]:
team_urls = []
for link in team_links:
    team_urls.append("https://www.espncricinfo.com/cricketers" + link)
team_urls

['https://www.espncricinfo.com/cricketers/team/afghanistan-40',
 'https://www.espncricinfo.com/cricketers/team/australia-2',
 'https://www.espncricinfo.com/cricketers/team/bangladesh-25',
 'https://www.espncricinfo.com/cricketers/team/england-1',
 'https://www.espncricinfo.com/cricketers/team/india-6',
 'https://www.espncricinfo.com/cricketers/team/ireland-29',
 'https://www.espncricinfo.com/cricketers/team/new-zealand-5',
 'https://www.espncricinfo.com/cricketers/team/pakistan-7',
 'https://www.espncricinfo.com/cricketers/team/south-africa-3',
 'https://www.espncricinfo.com/cricketers/team/sri-lanka-8',
 'https://www.espncricinfo.com/cricketers/team/west-indies-4',
 'https://www.espncricinfo.com/cricketers/team/zimbabwe-9']

In [8]:
# getting the players url
player_urls = []
driver = webdriver.Chrome()
# set up wait object
wait = WebDriverWait(driver, 10)

for url in team_urls:
    driver.get(url)

    # click on ODI span
    odi_span = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='ODI']")))
    ActionChains(driver).move_to_element(odi_span).click().perform()

    # wait for players to load
    time.sleep(10)

    # extract player links
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    players = soup.find_all("a", class_="ds-inline-flex ds-items-start ds-leading-none")
    for player in players:
        if "cricketers" in player['href']:
            player_urls.append("https://www.espncricinfo.com/" + player['href'])
    

# close the driver
driver.quit()

In [11]:
player_urls

['https://www.espncricinfo.com//cricketers/azmatullah-omarzai-819429',
 'https://www.espncricinfo.com//cricketers/fareed-ahmad-568136',
 'https://www.espncricinfo.com//cricketers/fazalhaq-farooqi-974175',
 'https://www.espncricinfo.com//cricketers/gulbadin-naib-352048',
 'https://www.espncricinfo.com//cricketers/hashmatullah-shahidi-440970',
 'https://www.espncricinfo.com//cricketers/ibrahim-zadran-921509',
 'https://www.espncricinfo.com//cricketers/mohammad-nabi-25913',
 'https://www.espncricinfo.com//cricketers/mujeeb-ur-rahman-974109',
 'https://www.espncricinfo.com//cricketers/najibullah-zadran-524049',
 'https://www.espncricinfo.com//cricketers/noor-ahmad-1182529',
 'https://www.espncricinfo.com//cricketers/rahmanullah-gurbaz-974087',
 'https://www.espncricinfo.com//cricketers/rahmat-shah-533956',
 'https://www.espncricinfo.com//cricketers/rashid-khan-793463',
 'https://www.espncricinfo.com//cricketers/riaz-hassan-1061090',
 'https://www.espncricinfo.com//cricketers/yamin-ahmadzai

In [9]:
#function to get the players info

def scrap_player_info(player_url):
    player_dict = {}
    player_response = requests.get(player_url)
    player_soup = BeautifulSoup(player_response.content,"html.parser")
    div_container = player_soup.find("div",class_ = "ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8")
    for div in player_soup.find("div",class_ = "ds-mb-4"):
        player_dict['Name'] = div.find('h1').text
        player_dict['Team'] = div.find_all('span',class_ = 'ds-text-comfortable-s')[0].text
        player_dict['Role'] = div.find_all('span',class_ = 'ds-text-comfortable-s')[1].text
        player_dict["International Span"] = div.find("div",class_ = "ds-bg-ui-fill-contrast ds-px-4 ds-py-2 ds-text-raw-grey06").span.text.split(":")[1]
    for div in div_container:
        label = div.find("p").text
        if label == "Age":
            age = div.find("span").text
            player_dict['Age'] = age
        elif label == "Batting Style":
            batting_style = div.find("span").text
            player_dict["Batting Style"] = batting_style
        elif label == "Bowling Style":
            bowling_style = div.find("span").text
            player_dict["Bowling Style"] = bowling_style
    return player_dict

In [62]:
#write players information into a csv file

def write_player_info_to_csv(url_list, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Team', 'Role','International Span', 'Age', 'Batting Style', 'Bowling Style']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for url in url_list:
            player_info = scrap_player_info(url)
            writer.writerow(player_info)
            
filename = 'player_info.csv'
write_player_info_to_csv(player_urls, filename)

In [36]:
#scrap the players stats
def scrap_stats(player_url):
    response_text = requests.get(player_url)
    response_soup = BeautifulSoup(response_text.content,"html.parser")
    odi_bat_rows = []
    odi_bowl_rows= []
    for h5 in response_soup.find_all("h5",class_ = "ds-text-title-subtle-s ds-font-medium ds-py-2 ds-ml-4 ds-text-typo ds-capitalize"):
        if 'batting' in h5.text.lower():
            table_tag = h5.find_all_next('table')[0] # Get the first table after the h5 tag
            batting_thead = table_tag.thead
            batting_cols = [th.text.strip() for th in batting_thead.find_all('th')]
            format_col_idx = batting_cols.index('Format')

            batting_tbody = table_tag.tbody
            batting_rows = batting_tbody.find_all('tr')

            for row in batting_rows:
                cols = row.find_all('td')
                if cols[format_col_idx].text.strip() == 'ODI':
                    odi_bat_rows.append([col.text.strip() for col in cols])
        if 'bowling' in h5.text.lower():
            table_tag = h5.find_all_next('table')[0] # Get the first table after the h5 tag
            bowling_thead = table_tag.thead
            bowling_cols = [th.text.strip() for th in bowling_thead.find_all('th')]
            format_col_idx = bowling_cols.index('Format')

            bowling_tbody = table_tag.tbody
            bowling_rows = bowling_tbody.find_all('tr')

            for row in bowling_rows:
                cols = row.find_all('td')
                if cols[format_col_idx].text.strip() == 'ODI':
                    odi_bowl_rows.append([col.text.strip() for col in cols])

    return odi_bat_rows[0], odi_bowl_rows[0]

In [37]:
# Function to scrape player stats and write them to CSV files
def scrape_and_write_stats(player_urls):
    # Open CSV files for writing
    with open('batting_stats.csv', 'w', newline='', encoding='utf-8') as batting_file, \
         open('bowling_stats.csv', 'w', newline='', encoding='utf-8') as bowling_file:
         
        # Create CSV writers
        batting_writer = csv.writer(batting_file)
        bowling_writer = csv.writer(bowling_file)
        
        # Write headers to CSV files
        batting_writer.writerow(['Player Name','Format','Matches', 'Innings', 'Not Outs','Runs','Highest Score','Average','BF','Strike Rate','100s','50s','4s','6s','Ct','St'])
        bowling_writer.writerow(['Player Name','Format','Matches','Innings','Balls','Runs','Wickets','BBI','BBM','Average','Economy','Strike Rate','4 Wicket Hauls','5 Wicket Hauls','10 Wicket Hauls'])
        
        # Loop through player URLs and scrape stats
        for url in player_urls:
            # Scrape batting and bowling stats for player
            batting_stats, bowling_stats = scrap_stats(url)
            text_obj = requests.get(url)
            soup = BeautifulSoup(text_obj.content,"html.parser")
            name_div = soup.find("div",class_ = "ds-mb-4")
            player_name = name_div.find("h1").text
            
            # Write batting stats to CSV
            batting_writer.writerow([player_name] + batting_stats)
            
            # Write bowling stats to CSV
            bowling_writer.writerow([player_name] + bowling_stats)
scrape_and_write_stats(player_urls)

# Scrap the players images

In [34]:
images_url = []
driver = webdriver.Chrome()
for url in player_urls:
    driver.get(url)
    wait = WebDriverWait(driver,3)
    images = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,"img")))
    player_images_url = [img.get_attribute("src") for img in images]
    if player_images_url[0]:
        images_url.append(player_images_url[0])
    else:
        images_url.append("-")
driver.quit()  

In [45]:
player_names = []
for url in player_urls:
    player_response = requests.get(url)
    player_soup = BeautifulSoup(player_response.content,"html.parser")
    div_container = player_soup.find("div",class_ = "ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8")
    for div in player_soup.find("div",class_ = "ds-mb-4"):
        player_names.append(div.find('h1').text)

['Azmatullah Omarzai',
 'Fareed Ahmad',
 'Fazalhaq Farooqi',
 'Gulbadin Naib',
 'Hashmatullah Shahidi',
 'Ibrahim Zadran',
 'Mohammad Nabi',
 'Mujeeb Ur Rahman',
 'Najibullah Zadran',
 'Noor Ahmad',
 'Rahmanullah Gurbaz',
 'Rahmat Shah',
 'Rashid Khan',
 'Riaz Hassan',
 'Yamin Ahmadzai',
 'Sean Abbott',
 'Ashton Agar',
 'Jason Behrendorff',
 'Alex Carey',
 'Pat Cummins',
 'Nathan Ellis',
 'Aaron Finch',
 'Cameron Green',
 'Josh Hazlewood',
 'Travis Head',
 'Josh Inglis',
 'Matthew Kuhnemann',
 'Marnus Labuschagne',
 'Mitchell Marsh',
 'Glenn Maxwell',
 'Ben McDermott',
 'Jhye Richardson',
 'Steven Smith',
 'Mitchell Starc',
 'Marcus Stoinis',
 'Mitchell Swepson',
 'David Warner',
 'Adam Zampa',
 'Afif Hossain',
 'Anamul Haque',
 'Ebadot Hossain',
 'Hasan Mahmud',
 'Litton Das',
 'Mahmudullah',
 'Mehidy Hasan Miraz',
 'Mosaddek Hossain',
 'Mushfiqur Rahim',
 'Mustafizur Rahman',
 'Najmul Hossain Shanto',
 'Nasum Ahmed',
 'Nurul Hasan',
 'Shakib Al Hasan',
 'Shoriful Islam',
 'Taijul Isl

In [73]:
player_info_with_images = list(zip(player_names,images_url))
with open('player_info.csv','r') as file:
    reader = csv.reader(file)
    player_data = list(reader)
player_data = player_data[1:]
df1 = pd.DataFrame(player_data,columns=['Name','Team','Playing Role','SPan','age','Batting Style','Bowling_style'])
df2 = pd.DataFrame(player_info_with_images,columns=['Name','Players images url'])
merged_df = pd.merge(df1,df2,on = "Name",how='left')

merged_df.to_csv('players_data_with_images.csv')